# Microgrid sizing optimization

Following the first example about [Microgrid performance simulation](Microgrid_Wind-Solar.ipynb)
using [Microgrids.py](https://github.com/Microgrids-X/Microgrids.py),
this notebook delves into **optimal microgrid sizing**.
This means finding the optimal size (rated power or energy) of each component by solving an optimization problem where the objective function is composed of Key Performance Indicators (KPIs) related to the economic and technical performance of the project.

Like in the previous example, the sizing is done for a microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*.

<img alt='schematic of a microgrid with a symbolic representation of the notion of sizing' src='./images/microgrid_sizing.png' style='height:18em'>

The main steps of the optimal sizing process are:
1. set up the problem (in particular loading project data like prices and time series)
2. run the optimization
3. analyze the results

## About the optimization formulation and process

### Optimization formulation

In this illustrative sizing example, the optimization objective (criterion) is:
- minimize the Levelized Costs of Electricity `lcoe` (€/kWh)
- while keeping the load shedding rate below some threshold (ex.: shedding rate ≤ 1%)

but other indicators could be used as well: rate of renewables above some threshold...

The above problem is a constrained optimization problem. Since we'll use unconstrained optimization solvers, we will reformulate it as: objective = lcoe + penalty for excess of shedding.

### Optimization process

This is a “blackbox” type of optimization problem: the optimization algorithm calls iteratively the Microgrid simulator with sizings to be tested until convergence (or reaching the maximum allowed number of iterations):

<img alt='schematic of microgrid sizing optimization process: iterations call of the simulator by optimizer' src='./images/MG_optim_diagram.png' style='height:18em'>

The code is meant for using the optimization solvers from [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html#optimization), but it could be adapted to other optimizers with minor changes (see remark below about the modularity of the code).

In [1]:
try: # Install microgrids package in JupyterLite (if run in JupyterLite)
    import piplite
    await piplite.install(['microgrids', 'ipywidgets'])
except ImportError:
    pass

In [2]:
import numpy as np
import scipy.optimize as opt
from matplotlib import pyplot as plt

import microgrids as mgs

## Load Microgrid project data

Loading parameters and time series for a Microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*. 
Values gathered from the [Microgrid_Wind-Solar.ipynb](Microgrid_Wind-Solar.ipynb) notebook.

In [3]:
%run -i Microgrid_Wind-Solar_data.py

Data definition for Microgrid with wind, solar, storage and generator...


## Setting up the cost function (criterion) to be optimized

The key coding objective is to **encapsulate** the microgrid simulator (`Microgrid.simulate` method) into an objective function that can be called by the optimization algorithm, that is which respects its expected interface (here [`minimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html#local-multivariate-optimization) or one of the [global optimizers](https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization) from `scipy.optimize`).

To increase the modularity which facilitates using optimization solvers others that Scipy's we implement the encapsulation by **3 nested functions**:

1. Simulation of Microgrid project described by a sizing vector `x` (vector input) → returns all simulation statistics
2. Extract KPIs of interest to build a multi-objective criterion: here lcoe and shedding rate
3. Combine these KPIs as one mono-objective criterion: here LCOE + penalty if shedding rate > shed_max
   - and match the interface expected by Scipy's optimizers

but if one cares more about compactness, this could be assembled into one single function.

### Wrapper of the Microgrid simulator

accept a sizing vector `x`, then create all `Microgrids.py` components and call `simulate`

In [4]:
def simulate_microgrid(x):
    """Simulate the performance of a Microgrid project of size `x`
    with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]
    
    Returns stats, costs
    """
    project = mgs.Project(lifetime, discount_rate, timestep, "€")
    # Split decision variables (converted MW → kW):
    power_rated_gen = x[0]*1000
    energy_rated_sto = x[1]*1000
    power_rated_pv = x[2]*1000
    power_rated_wind = x[3]*1000

    # Create components
    gen = mgs.DispatchableGenerator(power_rated_gen,
        fuel_intercept, fuel_slope, fuel_price,
        investment_price_gen, om_price_gen, lifetime_gen,
        load_ratio_min,
        replacement_price_ratio, salvage_price_ratio, fuel_unit)
    batt = mgs.Battery(energy_rated_sto,
        investment_price_sto, om_price_sto, lifetime_sto, lifetime_cycles,
        charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_ini,
        replacement_price_ratio, salvage_price_ratio)
    pv = mgs.Photovoltaic(power_rated_pv, irradiance,
        investment_price_pv, om_price_pv,
        lifetime_pv, derating_factor_pv,
        replacement_price_ratio, salvage_price_ratio)
    windgen = mgs.WindPower(power_rated_wind, cf_wind,
        investment_price_wind, om_price_wind,
        lifetime_wind,
        replacement_price_ratio, salvage_price_ratio)
    mg = mgs.Microgrid(project, Pload, gen, batt, {
        'Solar': pv,
        'Wind': windgen
    })

    # Launch simulation:
    stats, costs = mg.simulate()

    return stats, costs

Test of the simulator wrapper (on a baseline sizing):

In [5]:
# Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = np.array([power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind])/1000.
# run simulation:
stats, costs = simulate_microgrid(x_base)
x_base, costs.lcoe, costs.npc/1e6

(array([1.8, 5. , 3. , 0.9]), 0.22924812869928668, 21.890027729086526)

### Build the objective functions (criteria)

- first bi-objective function x ↦ (lcoe, shedding rate)(x)
- then wrapped into a mono objective x ↦ J(x) by using a penalty for the excess of shedding rate
  - and match the interface expected by NLopt's optimizers

In [6]:
def obj_multi(x):
    "Multi-objective criterion for microgrid performance: lcoe, shedding rate"
    stats, costs = simulate_microgrid(x)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate # in [0,1]
    return lcoe, shed_rate

In [7]:
def obj(x, shed_max, w_shed_max=1e5):
    """Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`
    
    load shedding penalty threshold `shed_max` should be in [0,1[
    """
    lcoe, shed_rate = obj_multi(x)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0:
        penalty = w_shed_max*over_shed
    else:
        penalty = 0.0
    return lcoe + penalty

### Tests the objective functions

Sizing being tested:
- baseline sizing from the simulation notebook: perfect quality of service (QoS) with zero load shedding
- baseline modified with a halved generator sizing: very good QoS with a bit of load shedding → not penalized
- small PV and small wind generators only: low LCOE (i.e. the production-only LCOE of these sources) but but extremely bad QoS → huge penalty

In [8]:
# Test:
shed_max = 0.01 # in [0,1]

x_shed = np.array([power_rated_gen/2, energy_rated_sto, power_rated_pv, power_rated_wind])/1000.
x_pv   = np.array([1e-4, 0., 500.,   0.])/1000. # 0-sized gen yields ZeroDivisionError in lifetime
x_wind = np.array([1e-4, 0.,   0., 500.])/1000.

print(f"Base. multi: {obj_multi(x_base)} mono: {obj(x_base, shed_max)}")
print(f"Shed. multi: {obj_multi(x_shed)} mono: {obj(x_shed, shed_max)}")
print(f"PV.   multi: {obj_multi(x_pv)} mono: {obj(x_pv, shed_max)}")
print(f"Wind. multi: {obj_multi(x_wind)} mono: {obj(x_wind, shed_max)}")

Base. multi: (0.22924812869928668, 0.0) mono: 0.22924812869928668
Shed. multi: (0.2067100016092461, 0.009602858175478346) mono: 0.2067100016092461
PV.   multi: (0.10149717606888833, 0.9235477392624462) mono: 91354.87542342069
Wind. multi: (0.10040273654193947, 0.7439572426735248) mono: 73395.82467008902


## Optimization

### Setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [10]:
Pload_max = np.max(Pload)

xmin = np.array([0., 0., 1e-3, 0.]) # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
x0 = np.array([1.0, 3.0, 3.0, 2.0]) * (Pload_max/1000)
xmax = np.array([1.2, 10.0, 10.0, 5.0]) * (Pload_max/1000)

Optionally ban some choices:

In [12]:
# Solar power forbidden (optional)
#x0[3] = 1e-3
#xmax[3] = 1e-3
# Wind power forbidden (optional)
#x0[4] = 0.
#xmax[4] = 0.

Check cost function on `xmin` and `xmax`

In [13]:
obj_multi(xmin), obj_multi(xmax)

((0.10149685980963591, 0.9998470957371233), (0.8229416738277804, 0.0))

### Wrapper of the optimization process

This is an optional step, but recommended to explore easily the impact of the many parameters taken by optimization algorithms.

See [optimization termination conditions](https://nlopt.readthedocs.io/en/latest/NLopt_Introduction/#termination-conditions) in NLopt documention for the meaning of `xtol_rel`

See https://docs.scipy.org/doc/scipy/reference/optimize.html#optimization

In [16]:
def optim_mg(x0, shed_max, algo='DIRECT', maxeval=1000, xtol_rel=1e-4, srand=1):
    """Optimize sizing of microgrid based on the `obj` function

    Parameters:
    - `x0`: initial sizing (for the algorithms which need them)
    - `shed_max`: load shedding penalty threshold (same as in `obj`)
    - `algo` could be one of 'DIRECT'...
    - `maxeval`: maximum allowed number of calls to the objective function,
      that is to the microgrid simulation
    - `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
    - `srand`: random number generation seed (for algorithms which use some stochastic search)
    
    Problem bounds are taken as the global variables `xmin`, `xmax`,
    but could be added to the parameters as well.
    """
    nx = len(x0) # number of optim variables
    bounds = opt.Bounds(xmin, xmax)
    if algo=='DIRECT':
        res = opt.direct(obj, bounds, args=(shed_max,), maxfun=maxeval)
    else:
        raise ValueError(f'Unsupported optimization algorithm {algo}')
    
    xopt = res.x
    return xopt, res, res.nfev

### Run optimization & analyze results

In [19]:
algo = 'DIRECT' # could be one of 'DIRECT'...
shed_max = 0.01 # in [0,1]
maxeval = 1000
xopt, res, numevals = optim_mg(x0, shed_max, algo, maxeval)

print(f'{algo} algo: {res.message} after {numevals} iterations.')
print(f'x*= {np.round(xopt*1000, decimals=1)}') # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
print(f'LCOE*: {lcoe_opt}', )
print(f'shed*: {shed_rate_opt}')

DIRECT algo: Number of function evaluations done is larger than maxfun=1000 after 1009 iterations.
x*= [ 910.6 1877.2 1550.2 1356.2]
LCOE*: 0.19126795464515603
shed*: 0.008861252247391888


---
First optim working OK

🚧 ***TO BE CONTINUED*** 🚧

optional local "polishing":

In [ ]:
algo_polish = :LN_SBPLX
xopt_polish, ret, numevals = optim_mg(xopt, shed_max, algo_polish)

@printf("%s polish: %s after %d iterations. \nx*=", algo_polish, ret, numevals)
println(round.(xopt_polish*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt_polish)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

Retrieve all performance statistics of the optimized sizing

In [ ]:
stats_opt, costs_opt = simulate_microgrid(xopt)
@printf("NPC*: %.2f M\$ (compared to %.2f M\$ in baseline)\n", costs_opt.npc/1e6, costs.npc/1e6)
@printf("rate of renewables: %.1f %%\n", stats_opt.renew_rate*100)
# Display all operation statistics
stats_opt

#### Summary of some results for 1% max shedding rate OUTDATED 💩

Global search algorithms:
- **CRS2**: 0.190 720 \$/kWh (with shedding ~ threshold).
  x* = [889.0, 2013.0, 1570.9, 1354.0] → *best global algorithm choice so far, although effect of seed not studied*
- **DIRECT**: 0.191 297 \$/kWh (with shedding < threshold) in 1000 iter.
  x* = [910.4, 1837.8, 1543.3, 1375.7] 
    - "polishing" with SBPLX: 0.190 650 \$/kWh (with shedding ~ threshold) in 207 extra iter.
      x* = [887.1, 1992.4, 1543.3, 1375.6]
    - increasing DIRECT maxeval to 10k doesn't bring much improvement (0.191 258 \$/kWh)
      compared to SBPLX polishing
- **ESCH**: 0.194 884 \$/kWh with shedding OK (with shedding < threshold) in 1000 iter.
  x* = [1028.1, 1769.6, 1420.5, 1365.1] → *much slower to converge than DIRECT or CRS*

Local search algorithms:
- **SUBPLX**: 0.190 630 \$/kWh with (with shedding ~ threshold) **in 479 iter** (<maxeval).
  x* = [888.6, 1873.5, 1558.5, 1355.9]
  - very competitive, but too sensitive to initialization `x0`?

#### Case with perfect quality of service (QoS)

Another optimization case with perfect quality of service (QoS), that is zero load shedding  

In [ ]:
algo = :GN_CRS2_LM # could be one of LN_SBPLX, GN_DIRECT, GN_CRS2_LM, GN_ESCH...
shed_max = 0.00 # in [0,1]
maxeval=1000
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

#### Performance (duration) of the optimization

About 11 s for 1k iterations, i.e. 11 ms/iter.

In [18]:
%timeit -n 1 -r 1 xopt, ret, numevals = optim_mg(x0, shed_max, 'DIRECT', 1000)

10.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Fact: optimization time relates to simulation time with Toptim ~ Tsim × neval

In [24]:
%timeit obj_multi(x_base) # with x0 it's slightly faster 10.5 → 9.5 ms

10.7 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Remark: for *some reason*, the simulator was at some point **twice slower** here compared to [Microgrid_Wind-Solar.ipynb](Microgrid_Wind-Solar.ipynb) (11 ms → 24 ms). BUT after restarting the kernel, it's back at 11 ms!

---
## *How we imagine the optimization interface in a future version...*

Desirable feature: *dynamically select* which variables are optimized (e.g. using `getattr` to dynamically access object fields...)

In [58]:
#mg0 = baseline microgrid project, with given price and time series data
opti_params = [ # list of component/variable name, initial, min, max values
    (['generator', 'power_rated'], 1000.0, 0., 2000.), 
    (['battery', 'energy_max'],    3000.0, 0., 6000.)
]
#optim_mg(mg0, opti_params)